# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     -------- ---------------------------- 235.5/990.0 kB 14.1 MB/s eta 0:00:01
     -------------------------- ----------- 686.1/990.0 kB 8.7 MB/s eta 0:00:01
     -------------------------------------- 990.0/990.0 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/373.5 kB ? eta -:--:--
     ------------------------------------- 373.5/373.5 kB 22.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
     -------------------------------------- 139.8/139.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.65
    Uninstalling langsmith-0.1.65:
      Successfully uninstalled langsmith-0.1.65
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      Successfully un

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires openai<2.0.0,>=1.24.0, but you have openai 1.14.2 which is incompatible.


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [7]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

language_template = """你是一位语言学领域的权威。你对世界上的语言及其历史有着深入的了解。
你的强项在于对不同语言之间的比较分析，以及对语言发展趋势的洞察。你通常会从语言的基本语法和词汇入手，逐步构建起对整个语言体系的理解，最终能够解答涉及多个语言的复杂问题。

这是一个问题： {input}"""

biology_template = """你是一位杰出的生物学权威。你对生命科学的各个分支有着透彻的理解。
你的专长在于将复杂的生物系统拆解为更易管理的部分，从而深入研究生物体的内在机制。你通常从单个细胞和基因的功能开始，逐步扩展到整个生态系统的相互作用，以此来解决涉及生物多样性和生态平衡的高级问题。

这是一个问题： {input}"""

it_template = """你是一位经验丰富的计算机科学专家。你对编程语言、算法和系统架构有着深刻的见解。
你擅长将复杂的技术挑战分解为更小的、可管理的任务，并且能够运用创新的解决方案来克服它们。你经常从代码的基本构建块开始，逐步构建复杂的软件应用和高效的系统架构，以此来解决当今世界中的信息技术难题。

这是一个问题： {input}"""

In [8]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": math_template,
    },
     {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": math_template,
    },
     {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": math_template,
    },
]

In [9]:
llm = OpenAI(api_key="sk-DBsuQMwk5L0Nhn28924910De59394b439a927d75E3C21c61", base_url="https://api.xiaoai.plus/v1")

In [12]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [13]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [14]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [15]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [16]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '汉语言: 适用于回答汉语言问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题']


In [17]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
汉语言: 适用于回答汉语言问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题


In [18]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [19]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
汉语言: 适用于回答汉语言问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题

<< INPUT >>
{input}

<< OU

In [22]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [23]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指处于热平衡状态的物体所发出的电磁辐射。它的特点是发射的辐射波长与温度有关，温度越高，发射的波长越短，辐射强度也越大。它是物体热辐射的基本现象，对于理解热力学和量子力学都具有重要意义。 '}


In [24]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n这个问题可以分解为以下几个部分：\n1. 找到第一个质数\n2. 将这个质数加一\n3. 判断加一后的数能否被3整除\n\n先回答这些组成部分，可以得到答案：\n1. 第一个质数是2\n2. 2加一为3\n3. 3能被3整除\n\n因此，第一个质数是2，使得这个质数加一能被3整除。'}


In [25]:
router_chain.verbose = True

In [26]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度的天体，它的重力场非常强大，甚至连光也无法逃离它的吸引力。它的存在是由于某些恒星在死亡过程中产生的，当其质量过大时，会发生引力坍缩，形成一个密度无穷大的点，即黑洞。它是宇宙中最神秘和最具挑战性的物质之一，也是当前物理学研究的热点之一。'}


In [28]:
print(
    chain.invoke(
        "请解释汉语中'文房四宝'指的是哪四样物品？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言: {'input': "请解释汉语中'文房四宝'指的是哪四样物品？"}
> Finished chain.
{'input': "请解释汉语中'文房四宝'指的是哪四样物品？", 'text': '\n\n"文房四宝"指的是毛笔、墨、纸和砚。毛笔是用来书写汉字的工具，墨是用来沾湿毛笔的颜料，纸是用来书写的载体，砚是用来研磨墨的工具。这四样物品一起被称为"文房四宝"，在中国传统文化中被视为书写和绘画必备的工具。它们也象征着中国古代文人的学问和修养。在现代，"文房四宝"也被用来形容文学艺术的四种基本元素。'}


In [29]:
print(
    chain.invoke(
        "在细胞分裂过程中，有丝分裂与减数分裂的主要区别是什么？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '在细胞分裂过程中，有丝分裂与减数分裂的主要区别是什么？'}
> Finished chain.
{'input': '在细胞分裂过程中，有丝分裂与减数分裂的主要区别是什么？', 'text': '\n\n答：有丝分裂和减数分裂都是细胞分裂的两种形式，它们的主要区别在于细胞分裂的目的和产生的细胞数目不同。\n有丝分裂是指细胞按照一定的步骤将自身复制成两个完全相同的细胞，这个过程也被称为有丝分裂的有丝期，最终产生两个染色体数目相同的细胞，这两个细胞都具有和原来细胞相同的染色体组成和基因信息。\n而减数分裂则是指细胞在复制后，通过两次分裂产生四个细胞，这个过程也被称为减数分裂的减数期，最终产生四个染色体数目减半的细胞，'}


In [30]:
print(
    chain.invoke(
        "解释一下什么是面向对象编程中的'继承'概念。"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': "Can you explain the concept of 'inheritance' in object-oriented programming?"}
> Finished chain.
{'input': "Can you explain the concept of 'inheritance' in object-oriented programming?", 'text': '\n\n当我们谈论面向对象编程时，继承是一个重要的概念。它是指一个类（也可以被称为父类或超类）可以通过创建一个新类（也可以被称为子类或派生类）来传递它的属性和方法。这意味着子类可以继承父类的所有特性，包括数据和行为，并且可以在不改变父类的情况下添加自己的特有特性。\n\n一个简单的比喻可以帮助我们理解继承的概念。想象一下一个家族，父母和他们的子女。父母拥有一些共同的特征，比如眼睛的颜色、身高等。子女也可以继承这些特征，但是他们也有自己独特的特征，比如头发的颜色、兴趣爱'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains